### **Web Scraping:** 
**El web scraping es el proceso de extracción automatizada de datos de sitios web. Utiliza programas de software para recorrer páginas web, analizar su estructura y extraer información específica de ellas. Este proceso puede incluir la descarga de contenido web, el análisis de HTML para identificar patrones de datos y la extracción de datos relevantes según ciertos criterios, como etiquetas HTML específicas, clases CSS o ubicaciones en la página. El objetivo del web scraping es obtener datos de manera eficiente y automatizada para su posterior análisis, almacenamiento o uso en otras aplicaciones.**

#### Importamos librerias necesarias para nuestro trabajo

In [20]:
import pandas as pd  # Importar la librería pandas para manipulación y análisis de datos
import numpy as np # Importar la librería numpy para la manipulacion de datos
import os  # Importar el módulo os para operaciones relacionadas con el sistema operativo
import re  # Importar el módulo re para operaciones de expresiones regulares
import requests  # Importar el módulo requests para realizar solicitudes HTTP
from bs4 import BeautifulSoup  # Importar BeautifulSoup desde el módulo bs4 para analizar HTML
import fastparquet  # Importar la biblioteca fastparquet para leer y escribir archivos Parquet

#### Segundo WebScraping: página web de **Global Petrol Price** https://es.globalpetrolprices.com/USA/

In [2]:
import requests
from bs4 import BeautifulSoup

#Colores
azul = "\33[1;36m" # Texto azul claro
gris = "\33[0;37m" # Texto gris
blanco = "\33[1;37m" # Texto blanco

def datos_game(url):
    """Devuelve información de un producto de la tienda game"""
    # Inicializamos el diccionario de salida
    d = {}
    try:
        # Cabeceras de la petición HTTP
        headers = {
            "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
        }
        # Realizamos la petición
        print(f'{azul}Realizando la petición:{blanco}{url}{gris}')
        req = requests.get(url, headers=headers, timeout=10)
        print(f'{azul}Código de respuesta...:{blanco}{req.status_code}{req.reason}{gris}')
        # Si la petición no fue correcta, devolvemos error
        req.raise_for_status()
        # Creamos un objeto BS4 a partir del código HTML
        soup = BeautifulSoup(req.text, "html.parser")
        # Nombre del producto
        nomeC = soup.find("div", id="graphPageLeft").find_all("th", class_="th")
        Conbustible = [img.text.strip() for img in nomeC]
        list_Names = [Conbustible[0] + ' en Galones'] + [Conbustible[1] + ' en Galones'] + [Conbustible[2] + ' en Galones'] + ['Precios de la electricidad por kWh en los '+ Conbustible[3]] + ['Precios de la electricidad por kWh en el '+ Conbustible[4]]
        d["Tipo de conbustible"] = list_Names  
        # Buscando dentro de soup los precios de los combustible
        priceC = soup.find("div", id="graphPageLeft").find_all("td", class_="value")
        pric = [img.text.strip() for img in priceC]
        price = [float(pric[4])*3.787] + [float(pric[7])*3.801] + [float(pric[10])*3.786] + [pric[16]] + [pric[19]]
        d["Precio"] = price
    except requests.exceptions.RequestException as e:
        print(f"{azul}Error al realizar la solicitud HTTP: {e}")
        return {"ERROR": f"Error al realizar la solicitud HTTP: {e}"}
    except Exception as e:
        print(f"{azul}Error inesperado: {e}")
        return {"ERROR": f"Error inesperado: {e}"}
    
    return d

# MAIN
if __name__ == "__main__":
    url = "https://es.globalpetrolprices.com/USA/"
    datos = datos_game(url)
    print(f'{blanco}IMPRIMIENDO DICCIONARIO')
    for clave, valor in datos.items():
        print(f'{azul}{clave.upper()}:{blanco}{valor}{gris}')


Realizando la petición:https://es.globalpetrolprices.com/USA/
Código de respuesta...:200OK
IMPRIMIENDO DICCIONARIO
TIPO DE CONBUSTIBLE:['Precios de la gasolina en Galones', 'Precios del diesel en Galones', 'Precios del etanol en Galones', 'Precios de la electricidad por kWh en los Hogares', 'Precios de la electricidad por kWh en el Negocio']
PRECIO:[3.8210829999999993, 4.0138560000000005, 2.881146, '0.172', '0.151']


In [16]:
# Visualizamos los datos de scrapeo
petrolpricedf = pd.DataFrame(datos)
petrolpricedf['Precio'] = petrolpricedf['Precio'].astype(float)
petrolpricedf

,Tipo de conbustible,Precio
0,Precios de la gasolina en Galones,3.821083
1,Precios del diesel en Galones,4.013856
2,Precios del etanol en Galones,2.881146
3,Precios de la electricidad por kWh en los Hogares,0.172000
4,Precios de la electricidad por kWh en el Negocio,0.151000


In [17]:
petrolpricedf.to_parquet("../0-DataSets/1-DatosScraping/petrolprice_2024.parquet")

#### Realizamos un `ETL` de los `archivos descargados` para tener un Dataset según proposito del proyecto.

In [22]:
# Datos proporcionados
PG = petrolpricedf["Precio"][0] # Asigna el valor de la primera fila de la columna 'Precio' en petrolpricedf a la variable PG
PE = petrolpricedf["Precio"][4] # Asigna el valor de la quinta fila de la columna 'Precio' en petrolpricedf a la variable PE

# Creación de un DataFrame con los datos proporcionados
data = {
    "Modelo": ["Toyota Camry", "Ford Crown Victoria", "Nissan NV200 Taxi", "Chevrolet Express", "Ram ProMaster City",
               "Tesla Model 3", "Chevrolet Bolt EV", "Nissan LEAF", "Hyundai Kona Electric", "Kia EV6"],
    "Tipo": ["Combustión", "Combustión", "Combustión", "Combustión", "Combustión", "Eléctrico", "Eléctrico", "Eléctrico", "Eléctrico", "Eléctrico"],
    "Precio (USD)": [25.295, 40.00, 31.00, 32.00, 26.495, 46.99, 26.595, 27.400, 34.00, 40.99],
    "Descuento E ($10MIL)": [25.295, 40.00, 31.00, 32.00, 26.495, 36.99, 16.595, 17.400, 24.00, 30.99],
    "Depreciación anual (%)": [15, 18, 17, 16, 15, 10, 11, 12, 13, 14],
    "Depreciacion 1er año": [3.79, 7.2, 5.27, 5.12, 3.97, 3.699, 1.83, 2.10, 3.12, 4.34],
    "Consumo mpg o kWh/100 millas": [39, 24.0, 24.0, 18.0, 24.0, 27.0, 29.0, 32.0, 25.8, 29.5],
    "Autonomía millas": [520, 350, 400, 450, 360, 358, 259, 226, 258, 239],
    "Precio (USD/galón o kWh)": [PG, PG, PG, PG, PG, PE, PE, PE, PE, PE], # Usa los valores de PG y PE
    "CO2 (g/milla)": [220, 370, 269, 434, 352, 0, 0, 0, 0, 0]
}

# Creación del DataFrame car_inversion_df con los datos proporcionados
car_inversion_df = pd.DataFrame(data)

# Multiplicar los valores de las columnas por 1000
car_inversion_df["Precio (USD)"] = car_inversion_df["Precio (USD)"] * 1000
car_inversion_df["Descuento E ($10MIL)"] = car_inversion_df["Descuento E ($10MIL)"] * 1000
car_inversion_df["Depreciacion 1er año"] = car_inversion_df["Depreciacion 1er año"] * 1000

# Redondear la columna "Precio (USD/galón o kWh)" a tres decimales
car_inversion_df["Precio (USD/galón o kWh)"] = round(car_inversion_df["Precio (USD/galón o kWh)"].astype(float), 3)

# Añadir una nueva columna "Millas/Año" al DataFrame car_inversion_df
car_inversion_df["Millas/Año"] = 125 * 365.25

# Calcular el "Gasto combustible anual" y redondear a dos decimales
car_inversion_df["Gasto combustible anual"] = np.where(car_inversion_df['Tipo'] == 'Combustión',
                                          car_inversion_df["Millas/Año"] * car_inversion_df["Precio (USD/galón o kWh)"] / car_inversion_df["Consumo mpg o kWh/100 millas"],
                                          car_inversion_df["Millas/Año"] * car_inversion_df["Precio (USD/galón o kWh)"] * car_inversion_df["Consumo mpg o kWh/100 millas"] / 100)
car_inversion_df["Gasto combustible anual"] = round(car_inversion_df["Gasto combustible anual"], 2)

# Calcular el "Gasto en 6 años / cambio de carro"
car_inversion_df["Gasto en 6 años / cambio de carro"] = car_inversion_df["Descuento E ($10MIL)"] + car_inversion_df["Gasto combustible anual"] * 6

# Calcular el "CO2 (Tn/6 años)"
car_inversion_df["CO2 (Tn/6 años)"] = car_inversion_df["Millas/Año"] * car_inversion_df["CO2 (g/milla)"] * 6 / 1000000

# Mostrar el DataFrame car_inversion_df
car_inversion_df


,Modelo,Tipo,Precio (USD),Descuento E ($10MIL),Depreciación anual (%),Depreciacion 1er año,Consumo mpg o kWh/100 millas,Autonomía millas,Precio (USD/galón o kWh),CO2 (g/milla),Millas/Año,Gasto combustible anual,Gasto en 6 años / cambio de carro,CO2 (Tn/6 años)
0,Toyota Camry,Combustión,25295.0,25295.0,15,3790.0,39.0,520,3.821,220,45656.25,4473.14,52133.84,60.266250
1,Ford Crown Victoria,Combustión,40000.0,40000.0,18,7200.0,24.0,350,3.821,370,45656.25,7268.86,83613.16,101.356875
2,Nissan NV200 Taxi,Combustión,31000.0,31000.0,17,5270.0,24.0,400,3.821,269,45656.25,7268.86,74613.16,73.689188
3,Chevrolet Express,Combustión,32000.0,32000.0,16,5120.0,18.0,450,3.821,434,45656.25,9691.81,90150.86,118.888875
4,Ram ProMaster City,Combustión,26495.0,26495.0,15,3970.0,24.0,360,3.821,352,45656.25,7268.86,70108.16,96.426000
5,Tesla Model 3,Eléctrico,46990.0,36990.0,10,3699.0,27.0,358,0.151,0,45656.25,1861.41,48158.46,0.000000
6,Chevrolet Bolt EV,Eléctrico,26595.0,16595.0,11,1830.0,29.0,259,0.151,0,45656.25,1999.29,28590.74,0.000000
7,Nissan LEAF,Eléctrico,27400.0,17400.0,12,2100.0,32.0,226,0.151,0,45656.25,2206.11,30636.66,0.000000
8,Hyundai Kona Electric,Eléctrico,34000.0,24000.0,13,3120.0,25.8,258,0.151,0,45656.25,1778.68,34672.08,0.000000
9,Kia EV6,Eléctrico,40990.0,30990.0,14,4340.0,29.5,239,0.151,0,45656.25,2033.76,43192.56,0.000000


Guardamos el dataset inversiones

In [23]:
car_inversion_df.to_parquet("../0-DataSets/3-DatosScrapingConETL/car_inversion_2023.parquet")